## TODO:

* TRAINING


* RECOMMENDED: (training speed-up) Precompute and cache each submodel’s softmax outputs once, store them on disk, and then train the meta‐learner on those saved vectors. This removes the four forward passes during each epoch.


* OPTIONAL: chatgpt recommends to not use the same train set for training ensemble model --> check this


* OPTIONAL: look at the difference between using one-hidden layer or not (BIG DIFFERENCE)


* OPTIONAL: improve trainig loop (with optional interrupt key and saving model)

In [35]:
import sys, os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
softmax = torch.nn.Softmax(dim=1)
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (
    top_k_accuracy_score,
    classification_report,
    confusion_matrix
)
import random
from pathlib import Path
from tqdm import tqdm
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation, SwinForImageClassification, SwinConfig
from typing import List, Tuple
import torch.nn.functional as F

In [2]:
# Compute absolute path to the `src/` folder
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH     = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

from utils import get_dataloaders, load_model, evaluate_model, print_metrics, plot_confusion_matrix, show_sample_predictions, plot_random_image_with_label_and_prediction

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

In [4]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device:", device)

Using device: mps


In [5]:
COUNTRIES = ['Albania', 'Andorra', 'Australia', 'Austria', 'Bangladesh', 'Belgium', 'Bhutan', 'Bolivia', 'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Chile', 'Colombia', 'Croatia', 'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador', 'Estonia', 'Eswatini', 'Finland', 'France', 'Germany', 'Greece', 'Guatemala', 'Hungary', 'Iceland', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Latvia', 'Lesotho', 'Lithuania', 'Luxembourg', 'Malaysia', 'Mexico', 'Montenegro', 'Netherlands', 'New Zealand', 'North Macedonia', 'Norway', 'Palestine', 'Peru', 'Poland', 'Portugal', 'Romania', 'Russia', 'Serbia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'United Arab Emirates', 'United Kingdom', 'United States']
num_classes = len(COUNTRIES)
project_root   = Path().resolve().parent

### Data

In [6]:
train_root = project_root/ "data" / "final_datasets" / "train"
train_loader = get_dataloaders(train_root, batch_size=32)

val_root = project_root/ "data" / "final_datasets" / "val"
val_loader = get_dataloaders(val_root, batch_size=32)

test_root = project_root/ "data" / "final_datasets" / "test"
test_loader = get_dataloaders(test_root, batch_size=32)

### Load models

In [44]:
def load_vit_model(model_path, device, num_classes=num_classes):
    """
    model_path   : Path or str to folder containing config.json & pytorch_model.bin
    device      : torch.device
    num_classes : int, number of output labels
    id2label    : dict mapping label IDs to string names
    label2id    : dict mapping string names to label IDs
    """


    label2id = {label: idx for idx, label in enumerate(COUNTRIES)}
    id2label = {idx: label for label, idx in label2id.items()}

    # 1) Load the original config, then override fields as needed
    config = SwinConfig.from_pretrained(model_path)
    config.num_labels = num_classes
    config.id2label   = id2label
    config.label2id   = label2id

    # 2) Instantiate model from your local checkpoint
    model = SwinForImageClassification.from_pretrained(
        pretrained_model_name_or_path=model_path,
        config=config,
        ignore_mismatched_sizes=True  # allows loading even if shapes changed
    )

    # 3) Move to device & set eval mode
    model.to(device).eval()
    return model

In [33]:
base_model = load_model(model_path=project_root / "models" / "resnet_finetuned_new" / "main.pth", device=device)
road = load_model(model_path=project_root / "models" / "resnet_finetuned_road_new" / "main.pth", device=device)
terrain = load_model(model_path=project_root / "models" / "resnet_finetuned_terrain_new" / "main.pth", device=device)
vegetation = load_model(model_path=project_root / "models" / "resnet_finetuned_vegetation_new" / "main.pth", device=device)

In [47]:
base_vit = load_vit_model(model_path=project_root / "models" / "swin_b_finetuned" / "swin_b_finetuned", device=device)
vit_road = load_vit_model(model_path=project_root / "models" / "swin_b_finetuned" / "swin_b_finetuned_road", device=device)
vit_terrain = load_vit_model(model_path=project_root / "models" / "swin_b_finetuned" / "swin_b_finetuned_terrain", device=device)
vit_vegetation = load_vit_model(model_path=project_root / "models" / "swin_b_finetuned" / "swin_b_finetuned_vegetation", device=device)

In [8]:
MODEL_NAME = "nvidia/segformer-b0-finetuned-cityscapes-768-768"

feature_extractor = SegformerFeatureExtractor.from_pretrained(MODEL_NAME)
seg_model = SegformerForSemanticSegmentation.from_pretrained(MODEL_NAME).eval()

/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:103: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [9]:
CITYSCAPES_LABELS = {
    0: 'road', 
    8: 'vegetation',  9: 'terrain'
}

TARGET_CLASSES = {'road','terrain','vegetation'}

## Ensemble

### Utils

In [10]:
# Utility to fetch softmax probs from a pretrained submodel
def get_probs(model, img_tensor, device):
    model.eval()
    with torch.no_grad():
        out = model(img_tensor.to(device))
        probs = nn.functional.softmax(out, dim=1).cpu().squeeze(0).numpy()
    return probs

In [11]:
def cache_submodel_outputs(base_ds, submodels, device, cache_dir):
    """
    Runs each submodel once over base_ds and writes:
      - feats.npy: shape (N, num_models * num_classes)
      - labels.npy: shape (N,)
    under cache_dir.
    """
    N = len(base_ds)
    num_classes = submodels['base'].fc.out_features
    num_models  = len(submodels)
    feats  = np.zeros((N, num_models * num_classes), dtype=np.float32)
    labels = np.zeros(N, dtype=np.int64)

    for i in range(N):
        print(f"Printing {i}/{N}")
        img, lbl = base_ds[i]                 # load and transform image
        labels[i] = lbl
        x = img.unsqueeze(0).to(device)

        vecs = []
        for name, m in submodels.items():
            m.eval()
            with torch.no_grad():
                out = m(x)
                p   = softmax(out).cpu().numpy().squeeze(0)
            vecs.append(p)
        feats[i] = np.concatenate(vecs)

    # Persist to disk once, not every epoch
    np.save(cache_dir / "feats.npy", feats)
    np.save(cache_dir / "labels.npy", labels)

In [12]:
class CachedEnsembleDataset(Dataset):
    def __init__(self, cache_dir):
        """
        Loads feats.npy and labels.npy once into memory.
        """
        self.feats  = np.load(cache_dir / "feats.npy")
        self.labels = np.load(cache_dir / "labels.npy")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        # returns (feature_vector, label) as torch tensors
        return torch.from_numpy(self.feats[i]), int(self.labels[i])

In [13]:
# Ensemble Network: one hidden layer
class EnsembleNet(nn.Module):
    def __init__(self, in_dim, hid_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, num_classes)
        )

    def forward(self, x):
        return self.net(x)

In [14]:
# Ensemble Network: one hidden layer
class EnsembleNet2(nn.Module):
    def __init__(self, in_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, num_classes)
        )

    def forward(self, x):
        return self.net(x)

In [15]:
def train_epoch(model, loader, loss_fn, optimizer, device, epoch, log_every=5):
    """
    Runs one epoch of training, printing updates every `log_every` batches.

    Args:
        model       (nn.Module):      the network to train
        loader      (DataLoader):     training data loader
        loss_fn     (callable):       loss function
        optimizer   (torch.optim.Optimizer)
        device      (torch.device)
        epoch       (int):            current epoch number (for prints)
        log_every   (int):            how many batches between prints

    Returns:
        avg_loss (float), avg_acc (float)
    """
    model.train()
    running_loss = 0.0
    running_correct = 0
    total_samples = 0

    for batch_idx, (imgs, labels) in enumerate(loader, start=1):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # Metrics
        preds = outputs.argmax(dim=1)
        batch_correct = (preds == labels).sum().item()
        batch_size = imgs.size(0)

        running_loss    += loss.item() * batch_size
        running_correct += batch_correct
        total_samples   += batch_size


    avg_loss = running_loss / total_samples
    avg_acc  = running_correct / total_samples
    return avg_loss, avg_acc


In [16]:
def eval_epoch(model, loader, loss_fn, device):
    model.eval()
    total_loss = total_correct = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            loss   = loss_fn(logits, y)
            preds  = logits.argmax(dim=1)
            total_correct += (preds==y).sum().item()
            total_loss    += loss.item() * X.size(0)
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

In [17]:
def train_with_early_stopping(
    model, train_loader, dev_loader,
    loss_fn, optimizer, device,
    ckpt_path,
    max_epochs=100, patience=5
):
    best_val_loss = float('inf')
    patience_ctr  = 0

    for epoch in range(1, max_epochs+1):
        # 1) Train epoch
        tr_loss, tr_acc = train_epoch(
            model, train_loader, loss_fn, optimizer, device, epoch
        )

        # 2) Eval on dev
        vl_loss, vl_acc = eval_epoch(model, dev_loader, loss_fn, device)
        print(f"Epoch {epoch}: train {tr_loss:.3f}/{tr_acc:.3f} | "
              f"val   {vl_loss:.3f}/{vl_acc:.3f}")

        # 3) Check for improvement
        if vl_loss < best_val_loss:
            best_val_loss = vl_loss
            patience_ctr  = 0
            torch.save(model.state_dict(), ckpt_path)
            print("  ↳ New best val loss; checkpoint saved.")
        else:
            patience_ctr += 1
            print(f"  ↳ No improvement. Patience {patience_ctr}/{patience}.")
            if patience_ctr >= patience:
                print("Early stopping triggered.")
                break

    # 4) Load best model before returning
    model.load_state_dict(torch.load(ckpt_path))
    return model

In [56]:
def extend_cache_with_vits(
    base_ds,                    # your CountryImageDataset over the same split
    old_cache_dir: Path,        # directory containing feats.npy & labels.npy
    vit_submodels: dict,        # {'base_vit': model, 'vit_road': m2, ...}
    device,
    new_cache_dir: Path=None    # where to write the extended cache
):
    """
    Reads old feats (shape N×(4*C)) and labels.
    Runs each ViT on every image once to get softmax vectors (C dims).
    Concats them to produce N×((4+4)*C) feats, saves to new cache.
    """
    # 1) Load old cache & labels
    feats_old = np.load(old_cache_dir / "feats.npy")   # shape [N, 4*C]
    labels    = np.load(old_cache_dir / "labels.npy")  # shape [N,]
    N, old_dim = feats_old.shape
    C = old_dim // 4   # number of classes

    # 2) Prepare container for new vit feats
    num_vits = len(vit_submodels)
    vit_feats = np.zeros((N, num_vits * C), dtype=np.float32)

    # 3) DataLoader to iterate base_ds in batch (optional speed)
    dl = DataLoader(base_ds, batch_size=32, shuffle=False, num_workers=4)
    start = 0

    count = 0
    for batch_imgs, _ in dl:
        print(count)
        count += 1
        bsz = batch_imgs.size(0)
        end = start + bsz

        # Compute each Vit’s probabilities
        batch_feats = []
        for name, m in vit_submodels.items():
            m.to(device).eval()
            with torch.no_grad():
                out = m(batch_imgs.to(device)).logits     # HuggingFace returns logits
                p   = softmax(out).cpu().numpy()  # shape [bsz, C]
            batch_feats.append(p)

        # Concatenate this batch’s vit vectors: [bsz, num_vits*C]
        vit_feats[start:end] = np.concatenate(batch_feats, axis=1)
        start = end

    assert end == N, "Dataset size mismatch"

    # 4) Build extended feats
    feats_new = np.concatenate([feats_old, vit_feats], axis=1)  # shape [N, 8*C]

    # 5) Save to cache directory
    if new_cache_dir is None:
        new_cache_dir = old_cache_dir
    new_cache_dir.mkdir(exist_ok=True, parents=True)
    np.save(new_cache_dir / "feats.npy", feats_new)
    np.save(new_cache_dir / "labels.npy", labels)

    print(f"Extended cache saved to {new_cache_dir} with shape {feats_new.shape}")

### Build cache

In [ ]:
submodels = {
    'base':     base_model,
    'road':     road,
    'terrain':  terrain,
    'vegetation': vegetation,
    'base_vit':    base_vit,
    'vit_road':    vit_road,
    'vit_terrain': vit_terrain,
    'vit_vegetation': vit_vegetation
}

vit_submodels = {
  'base_vit':    base_vit,
  'vit_road':    vit_road,
  'vit_terrain': vit_terrain,
  'vit_vegetation': vit_vegetation
}

In [51]:
train_cache_dir = project_root / "cache" / "cache_train"
dev_cache_dir = project_root / "cache" / "cache_dev"
test_cache_dir = project_root / "cache" / "cache_test"

new_train_cache_dir = project_root / "new_cache" / "cache_train"
new_dev_cache_dir = project_root / "new_cache" / "cache_dev"
new_test_cache_dir = project_root / "new_cache" / "cache_test"

In [ ]:
# Train split caching
train_cache_dir.mkdir(exist_ok=True)
cache_submodel_outputs(train_loader.dataset, submodels, device, train_cache_dir)

# Dev split caching
dev_cache_dir.mkdir(exist_ok=True)
cache_submodel_outputs(val_loader.dataset, submodels, device, dev_cache_dir)

# Test split caching
test_cache_dir.mkdir(exist_ok=True)
cache_submodel_outputs(test_loader.dataset, submodels, device, test_cache_dir)

Printing 0/23760
Printing 1/23760
Printing 2/23760
Printing 3/23760
Printing 4/23760
Printing 5/23760
Printing 6/23760
Printing 7/23760
Printing 8/23760
Printing 9/23760
Printing 10/23760
Printing 11/23760
Printing 12/23760
Printing 13/23760
Printing 14/23760
Printing 15/23760
Printing 16/23760
Printing 17/23760
Printing 18/23760
Printing 19/23760
Printing 20/23760
Printing 21/23760
Printing 22/23760
Printing 23/23760
Printing 24/23760
Printing 25/23760
Printing 26/23760
Printing 27/23760
Printing 28/23760
Printing 29/23760
Printing 30/23760
Printing 31/23760
Printing 32/23760
Printing 33/23760
Printing 34/23760
Printing 35/23760
Printing 36/23760
Printing 37/23760
Printing 38/23760
Printing 39/23760
Printing 40/23760
Printing 41/23760
Printing 42/23760
Printing 43/23760
Printing 44/23760
Printing 45/23760
Printing 46/23760
Printing 47/23760
Printing 48/23760
Printing 49/23760
Printing 50/23760
Printing 51/23760
Printing 52/23760
Printing 53/23760
Printing 54/23760
Printing 55/23760
Pr

In [57]:
extend_cache_with_vits(train_loader.dataset, train_cache_dir, vit_submodels, device, new_train_cache_dir)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [59]:
extend_cache_with_vits(val_loader.dataset, dev_cache_dir, vit_submodels, device, new_dev_cache_dir)
extend_cache_with_vits(test_loader.dataset, test_cache_dir, vit_submodels, device, new_test_cache_dir)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
Extended cache saved to /Users/michelangelonardi/Desktop/Università/Master/Bocconi Master/Year 1/Semester2/Computer Vision & Image processing/Final - project/rainbot/new_cache/cache_dev with shape (2970, 528)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
Extended cache saved to /Users/michelangelonardi/Desktop/Università/Master/Bocconi Master/Year 1/Semester2/Computer Vision & Image processing/Final - project/rainbot/new_cache/cache_test with shape (2970, 528)


### Training

In [61]:
train_cached_ds = CachedEnsembleDataset(new_train_cache_dir)
train_cached_loader = DataLoader(train_cached_ds, batch_size=32, shuffle=True, num_workers=0)

dev_cached_ds = CachedEnsembleDataset(new_dev_cache_dir)
dev_cached_loader = DataLoader(dev_cached_ds, batch_size=32, shuffle=False, num_workers=0)

test_cached_ds = CachedEnsembleDataset(new_test_cache_dir)
test_cached_loader = DataLoader(test_cached_ds, batch_size=32, shuffle=False, num_workers=0)

In [76]:
# Instantiate meta‐model
project_root = Path().resolve().parent

hid_dim=128
epochs=100
lr=1e-3

in_dim = num_classes * len(submodels)
model = EnsembleNet(in_dim, hid_dim, num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
opt     = optim.Adam(model.parameters(), lr=lr)

best_val_loss = float('inf')
ckpt_path = project_root / "models" / "ensemble" / "main.pth"

In [78]:
model = train_with_early_stopping(
    model,
    train_cached_loader,
    dev_cached_loader,
    loss_fn,
    opt,
    device,
    ckpt_path,
    max_epochs=100,
    patience=5
)

Epoch 1: train 0.394/0.895 | val   1.678/0.612
  ↳ New best val loss; checkpoint saved.
Epoch 2: train 0.375/0.899 | val   1.698/0.618
  ↳ No improvement. Patience 1/5.
Epoch 3: train 0.360/0.904 | val   1.732/0.615
  ↳ No improvement. Patience 2/5.
Epoch 4: train 0.346/0.905 | val   1.748/0.617
  ↳ No improvement. Patience 3/5.
Epoch 5: train 0.333/0.908 | val   1.774/0.615
  ↳ No improvement. Patience 4/5.
Epoch 6: train 0.322/0.910 | val   1.793/0.614
  ↳ No improvement. Patience 5/5.
Early stopping triggered.


/var/folders/d5/5853p9zd6pvbjb8vz92bhmnc0000gn/T/ipykernel_42652/589457112.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


In [80]:
# test 
model.eval()

test_loss, test_acc = eval_epoch(
    model, 
    test_cached_loader,  # DataLoader over CachedEnsembleDataset for test
    loss_fn, 
    device
)

print(f"Test   Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")


Test   Loss: 1.5744 | Test Acc: 0.6323


## Weights visualization

In [67]:
import numpy as np

def inspect_ensemble_weights(model, submodel_names, COUNTRIES):
    """
    Prints the L2‐norm of each [hid_dim × num_classes] weight block
    corresponding to each submodel in the meta‐learner's first layer.

    Args:
      model           : your trained EnsembleNet()
      submodel_names  : list of strings, e.g.
                        ['res_base','res_road','res_terrain','res_veg',
                         'vit_base','vit_road','vit_terrain','vit_veg']
      COUNTRIES       : list of country names used as labels
    """
    # 1) Extract the first Linear layer's weight matrix
    W1 = model.net[0].weight.data.cpu().numpy()  
    num_classes = len(COUNTRIES)
    hid_dim     = W1.shape[0]

    # 2) Sanity check
    expected_blocks = len(submodel_names)
    if W1.shape[1] != expected_blocks * num_classes:
        raise ValueError(
            f"Weight matrix has width {W1.shape[1]}, but expected "
            f"{expected_blocks}×{num_classes}={expected_blocks*num_classes}"
        )

    # 3) Slice into blocks and compute norms
    block_norms = []
    for i, name in enumerate(submodel_names):
        start = i * num_classes
        end   = (i+1) * num_classes
        block = W1[:, start:end]            # shape (hid_dim, num_classes)
        norm  = np.linalg.norm(block)       # L2 norm over all entries
        block_norms.append((name, norm))

    # 4) Print neatly
    print("First‐layer L2 norms per submodel block:")
    for name, norm in block_norms:
        print(f"  {name:12s}: {norm:.2f}")


In [68]:
submodel_names = [
    'res_base', 'res_road', 'res_terrain', 'res_veg',
    'vit_base','vit_road','vit_terrain','vit_vegetation'
]

inspect_ensemble_weights(model, submodel_names, COUNTRIES)


First‐layer L2 norms per submodel block:
  res_base    : 171.35
  res_road    : 69.65
  res_terrain : 40.79
  res_veg     : 51.57
  vit_base    : 97.93
  vit_road    : 22.57
  vit_terrain : 16.52
  vit_vegetation: 46.52


In [75]:
inspect_ensemble_weights(model, submodel_names, COUNTRIES)


First‐layer L2 norms per submodel block:
  res_base    : 24.00
  res_road    : 9.96
  res_terrain : 4.78
  res_veg     : 8.39
  vit_base    : 11.99
  vit_road    : 8.88
  vit_terrain : 9.04
  vit_vegetation: 8.97


In [46]:
inspect_ensemble_weights(model, COUNTRIES)

First‐layer L2 norms per submodel block:
  base: 25.40
  road: 10.85
  terrain: 6.12
  veg: 10.01


In [66]:
inspect_ensemble_weights(model, COUNTRIES)

First‐layer L2 norms per submodel block:
  base: 171.35
  road: 69.65
  terrain: 40.79
  veg: 51.57
